**Classifying Movie Reviews as Positive or Negative using the IMDB Dataset**

In [ ]:
#importing the required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#loading the dataset
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000) #keeping the top 10,000 most frequently occuring words in the training data

17465344/17464789 [==============================] - 0s 0us/step


***Exploratory Data Analysis*** :

In [ ]:
m_train = train_data.shape[0]
m_test = test_data.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print("train_data shape: "+ str(train_data.shape))
print("test_data shape: "+ str(test_data.shape))

Number of training examples: 25000
Number of testing examples: 25000
train_data shape: (25000,)
test_data shape: (25000,)


In [ ]:
#example
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
#example
print(train_labels[0])

1


In [ ]:
#example
len(train_data[0]), len(train_data[1])

(218, 189)

In [ ]:
#to decode the review back to english words

word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0     #padding
word_index["<START>"] = 1   #start of the sequence 
word_index["<UNK>"] = 2     # unknown
word_index["<UNUSED>"] = 3  

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

***Data Preprocessing*** : 

In [ ]:
#to standardize the lengths of the movie reviews

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [ ]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [ ]:

size_vocab = 10000

model = models.Sequential()

#takes the integer encoded vocabulary and looks up the embedding vector for each word index
model.add(layers.Embedding(size_vocab, 16))

#pooling layer to perform average pooling
model.add(layers.GlobalAveragePooling1D())

#dense layer using relu as activation function and 16 hidden units
model.add(layers.Dense(16, activation='relu'))

#dense layer using sigmoid as activation function for binary classification 
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


***Compiling the Model***

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])


In [ ]:
#splitting the training data into training and validation sets
x_val = train_data[:10000]
partial_x_train = train_data[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

***Training the Model***

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 24ms/step - loss: 0.6922 - acc: 0.5607 - val_loss: 0.6905 - val_acc: 0.6268
Epoch 2/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6874 - acc: 0.7213 - val_loss: 0.6841 - val_acc: 0.7172
Epoch 3/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6772 - acc: 0.7577 - val_loss: 0.6708 - val_acc: 0.7432
Epoch 4/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6588 - acc: 0.7696 - val_loss: 0.6492 - val_acc: 0.7414
Epoch 5/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6302 - acc: 0.7823 - val_loss: 0.6182 - val_acc: 0.7762
Epoch 6/40
30/30 [==============================] - 1s 19ms/step - loss: 0.5918 - acc: 0.8043 - val_loss: 0.5793 - val_acc: 0.7974
Epoch 7/40
30/30 [==============================] - 1s 18ms/step - loss: 0.5465 - acc: 0.8238 - val_loss: 0.5367 - val_acc: 0.8145
Epoch 8/40
30/30 [==============================] - 1s 19ms/step - loss: 0.4991 - a

In [ ]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

***Plotting the Curves***

In [ ]:
#Training vs Validation Loss Curve
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(acc) +1)

plt.plot(epochs,loss_values,'r',label='Training Loss')
plt.plot(epochs,val_loss_values,'b',label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

In [ ]:
#Training vs Validation Accuracy Curve
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs,acc,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'b',label = 'Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

***Evaluating on Test Data***

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)
print("Test Loss: " +str(results[0]))
print("Test Accuracy: " +str(results[1]))
p = round(results[1]*100,2)
print("Test Accuracy in %: " +str(p) +"%")

In [ ]:
model.predict(test_data)